In [2]:
# RWalker Oct. 15 Chord Candy Prototype Update
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import math
from nested_lookup import nested_lookup
import json
from music21 import *

In [3]:
# Load in dictionary for use

with open('chordDictionary.json', 'r') as dictOpen:
    nameToNum = json.load(dictOpen)

In [4]:
# Splits a 6 digit triad input input into individual note values
def chordIntSplit (chordInt, interval=2):
    chordInt = str(chordInt)
    split = np.array([])
    for k in range(0, len(chordInt), interval):
        split = np.append(split, int(chordInt[k:k + interval]))
    return split.astype(int)


# Reverses chordIntSplit, will combine the individual note values back into a single integer
def chordIntCombine (input):
    combine = int(str(input[0]) + str(input[1]) + str(input[2]))
    return combine

In [5]:
# Starting chord input: 
inputChord = 'F Major 2nd Inversion'
inputNum = nested_lookup(inputChord, nameToNum)
inputSplit = np.array(chordIntSplit(inputNum[0]))
# Print out MIDI numbers from user input
#print(inputSplit)

destinationChord = 'A#/Bb Major'
destinationChord = destinationChord.replace("b", "-")
destinationNums = nested_lookup(key = destinationChord, document = nameToNum, wild = True, with_keys= False)
#print(destinationNums)

lookupArray = np.zeros(9)
magArray = np.zeros(9)
nameArray = np.zeros(9, dtype=object)
num = 0

# Make list of inversions
for n in destinationNums[0]:
    #print(str(n))
    nIndex = num*3
    nameArray[nIndex:(nIndex+3)] = n
    num = num+1

for n in range(0, len(destinationNums)-1):
    arrayIndex = n*3
    # Split destination chord into separate note values for comparison
    destTest = np.array(chordIntSplit(destinationNums[n+1]))
    lookupArray[arrayIndex] = chordIntCombine(destTest)
    magArray[arrayIndex] = np.sum(np.abs(destTest - inputSplit))
    #print(destTest)

    # Look at octave above and octave below
    upOct = destTest + 12
    lookupArray[arrayIndex+1] = chordIntCombine(upOct)
    magArray[arrayIndex+1] = np.sum(np.abs(upOct - inputSplit))
    #print(upOct)
    downOct = destTest - 12
    
    lookupArray[arrayIndex+2] = chordIntCombine(downOct)
    magArray[arrayIndex+2] = np.sum(np.abs(downOct - inputSplit))
    #print(downOct)

combArr = np.column_stack((magArray, nameArray))

minVal = np.argmin(magArray)
destinationName = combArr[minVal][1]
destinationName = destinationName.replace("-", "b")
outputSplit = chordIntSplit(int(lookupArray[minVal]))


print("Input Notes: ", inputSplit.astype(int))
print("Output Notes: ", outputSplit.astype(int))
print("Magnitude of Change: ", int(combArr[minVal][0]), "Semitones")
print("Input Chord Name: ", inputChord)
print("Destination Chord Name: ", destinationName)

Input Notes:  [48 53 57]
Output Notes:  [50 53 58]
Magnitude of Change:  3 Semitones
Input Chord Name:  F Major 2nd Inversion
Destination Chord Name:  A#/Bb Major 1st Inversion


In [6]:
def getBestInversion(inChord, destChord):
    if type(inChord) is str:
        r = nested_lookup(inChord, nameToNum)
        try:
            inputNum = r[0]
        except IndexError:
            z = 'inChord has an invalid string (string is case sensitive)'
            return z
    elif type(inChord) is int:
        if len(str(inChord)) != 6:
            z = 'inChord vector spelling requires a six digit integer'
            return z
        else:
            inputNum = inChord
    else:
        z = 'inChord must be a chord vector integer or a string with a chord name found within nameToNum'
        return z
    try:
        inputSplit = np.array(chordIntSplit(inputNum))
    except ValueError:
        z = 'inChord taken as a string must include note, type, and inversion'
        return z
    # print(inputSplit)
    if type(destChord) is str:
        pass
    else:
        z = 'destChord must be a string, note name followed by quality'
        return z
    destChord = destChord.replace("b", "-")
    destNums = nested_lookup(key = destChord, document = nameToNum, wild = True, with_keys= False)

    lookupArray = np.zeros(9)
    magArray = np.zeros(9)
    nameArray = np.zeros(9, dtype=object)
    num = 0

    #Make list of inversions
    for n in destNums[0]:
        nIndex = num*3
        nameArray[nIndex:(nIndex+3)] = n
        num += 1

    for n in range(0, len(destNums)-1):
        arrayIndex = n*3
        # Split destination chord into separate note values for comparison
        destTest = np.array(chordIntSplit(destNums[n+1]))
        lookupArray[arrayIndex] = chordIntCombine(destTest)
        magArray[arrayIndex] = np.sum(np.abs(destTest - inputSplit))

    # Look at octave above and octave below
        upOct = destTest + 12
        lookupArray[arrayIndex+1] = chordIntCombine(upOct)
        magArray[arrayIndex+1] = np.sum(np.abs(upOct - inputSplit))
        
        downOct = destTest - 12
        lookupArray[arrayIndex+2] = chordIntCombine(downOct)
        magArray[arrayIndex+2] = np.sum(np.abs(downOct - inputSplit))


    combArr = np.column_stack((magArray, nameArray))

    minVal = np.argmin(magArray)
    destinationName = combArr[minVal][1]
    destinationName = destinationName.replace("-", "b")
    outputSplit = chordIntSplit(int(lookupArray[minVal]))
    magnitudeCalc = int(combArr[minVal][0])

    outputPitch = np.array([], dtype=str)
    for i in range(len(outputSplit)):
        np.append(outputPitch, str(pitch.Pitch(outputSplit.astype(int)[i])))

    print("Input Notes: ", inputSplit)
    print("Output Notes: ", outputSplit)
    print("Magnitude of Change: ", magnitudeCalc, "Semitone(s)")
    if type(inChord) is str:
        print("Input Chord Name: ", inChord)
    else:
        ic = 'TRY TO FIND THE CHORD IN THE DICTIONARY HERE'
    print("Destination Chord Name: ", destinationName)
    return destinationName, outputSplit.tolist(), magnitudeCalc, outputPitch



In [7]:
x = getBestInversion('D Augmented Root Position','F#/Gb Major')
print(x)

Input Notes:  [50 54 58]
Output Notes:  [49 54 58]
Magnitude of Change:  1 Semitone(s)
Input Chord Name:  D Augmented Root Position
Destination Chord Name:  F#/Gb Major 2nd Inversion
('F#/Gb Major 2nd Inversion', [49, 54, 58], 1, array([], dtype='<U1'))


In [8]:
print(x)

('F#/Gb Major 2nd Inversion', [49, 54, 58], 1, array([], dtype='<U1'))


In [9]:
x = pitch.Pitch(61)
print(x)

C#4


In [10]:
outputPitch = np.array([], dtype=str)
for i in range(len(outputSplit)):
    np.append(outputPitch, str(pitch.Pitch(outputSplit.astype(int)[i])))

In [11]:
pitch.Pitch(int(outputSplit[1]))

<music21.pitch.Pitch F3>

In [12]:
outputSplit.astype(int)

array([50, 53, 58])

In [13]:
outputPitch

array([], dtype='<U1')